## Import related API

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from google.colab import drive

In [23]:
# drive.mount('/content/gdrive')

## Read file

In [24]:
def read(path):
    return pd.read_csv(path)

Read file from Google Drive

In [25]:
path = "WeelyFinalData.csv"
finalData = read(path)

Drop Date indicacy from the data

In [26]:
# train = finalData.drop(["Date"], axis=1)
train = finalData

## Visualization for raw data

In [27]:
# import os 
# os.chdir("/content/gdrive/My Drive/Colab Notebooks")

In [28]:
def show_raw_visualization(data):

    fig, axes = plt.subplots(
        nrows=1, ncols=2, figsize=(15, 20), dpi=80, facecolor="w", edgecolor="k"
    )

    for i in range(data.shape[1]):
        t_data = data.iloc[:,i]
        ax = t_data.plot(
            ax=axes[i // 2, i % 2],
            color="black",
            title=data.columns[i]
        )
        
    plt.tight_layout()
    plt.savefig('/content/gdrive/My Drive/Colab Notebooks/raw_data.png')


# show_raw_visualization(train)

Dismiss presented by scientific notation

In [29]:
print(train)

        CCSP     CDCS     ADCS      CDWS       GL      NDCS  NRBCON     SL  \
0    58960.0  7864.00  2126.40  423475.0  1737.46  18794.80    2.63  34.09   
1    57595.0  7654.70  2116.47  411505.0  1779.54  18253.00    2.66  34.51   
2    57775.0  7625.90  2108.35  401500.0  1753.61  17876.40    2.55  33.28   
3    56015.0  7248.95  2017.10  396255.0  1689.11  17287.80    2.51  31.79   
4    57365.0  7684.15  2073.60  388955.0  1733.17  17230.05    2.56  32.40   
..       ...      ...      ...       ...      ...       ...     ...    ...   
475  57328.0  7691.95  2010.93  149685.0  1848.03  16173.71    1.26  24.26   
476  57620.0  7762.19  2024.22  146080.0  1838.77  17263.05    1.31  24.03   
477  58530.0  7846.47  2019.03  125365.0  1873.15  17198.90    1.37  25.68   
478  58332.0  7804.34  2008.50  116117.5  1876.39  16887.75    1.38  25.90   
479  58070.0  7777.33  1981.77  108925.0  1896.63  16608.83    1.41  26.43   

        ZSCS  EURUSD  USDCLP  USDCNY  USDPEN  USInflation  Chin

## Add lag time from 1~4 for CCSP

In [30]:
def buildTrain(train, pastWeek=1, futureWeek=4, defaultWeek=1):
    X_train, Y_train = [], []
    for i in range(train.shape[0]-futureWeek-pastWeek):
        X = np.array(train.iloc[i:i+defaultWeek])
        X = np.append(X,train.iloc[i+defaultWeek:i+pastWeek,:9])
        X_train.append(X.reshape(X.size))
        Y_train.append(np.array(train.iloc[i+pastWeek:i+pastWeek+futureWeek]["CCSP"]))
    return np.array(X_train), np.array(Y_train)

In [31]:
a = np.array([[0,1,2],[2,3,4]])
a[1]

array([2, 3, 4])

## Test

In [32]:
np.set_printoptions(suppress=True)

In [33]:
for i in range(4):
    X1_train, Y1_train= buildTrain(train, pastWeek=i+1)
    print("X1_Training_shape:{}".format(X1_train.shape))
    print("Y1_Training_shape:{}".format(Y1_train.shape))
    print("X[0]:{}".format(X1_train[0]))
    print("Y[0]:{}".format(Y1_train[0]))
    print("-"*30)

X1_Training_shape:(475, 15)
Y1_Training_shape:(475, 4)
X[0]:[ 58960.     7864.     2126.4  423475.     1737.46  18794.8       2.63
     34.09   1941.7       1.37    494.29      6.35      2.71      3.42
      4.46]
Y[0]:[57595. 57775. 56015. 57365.]
------------------------------
X1_Training_shape:(474, 24)
Y1_Training_shape:(474, 4)
X[0]:[ 58960.     7864.     2126.4  423475.     1737.46  18794.8       2.63
     34.09   1941.7       1.37    494.29      6.35      2.71      3.42
      4.46  57595.     7654.7    2116.47 411505.     1779.54  18253.
      2.66     34.51   1932.15]
Y[0]:[57775. 56015. 57365. 58135.]
------------------------------
X1_Training_shape:(473, 33)
Y1_Training_shape:(473, 4)
X[0]:[ 58960.     7864.     2126.4  423475.     1737.46  18794.8       2.63
     34.09   1941.7       1.37    494.29      6.35      2.71      3.42
      4.46  57595.     7654.7    2116.47 411505.     1779.54  18253.
      2.66     34.51   1932.15  57775.     7625.9    2108.35 401500.
   1753.61 

##Split the data to Training set & Test set

In [35]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))

##Building LSTM
[Reference](https://ithelp.ithome.com.tw/articles/10206312)

import related package

In [40]:
import tensorflow.keras as keras
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense, LSTM, Dropout, TimeDistributed, LeakyReLU, GRU, BatchNormalization
from keras.layers.core import RepeatVector
from keras.callbacks import EarlyStopping

In [41]:
def buildTwoLayerNN(training_data_shape):

    keras.backend.clear_session()
    regressor = Sequential()
    regressor.add(Dense(units=20, activation ="relu", input_shape=(training_data_shape,1)))
    regressor.add(Dense(units=20, activation ="relu"))
    regressor.add(Dense(units=1))

    regressor.compile(optimizer="adam", loss='mean_squared_error', metrics=["accuracy"])
    regressor.summary()

    return regressor

In [42]:
def buildLSTMModel(training_data_shape):

    keras.backend.clear_session()

    regressor = Sequential()

    regressor.add(LSTM(units=20, return_sequences=True, activation = LeakyReLU(alpha=0.1), input_shape=(training_data_shape,1)))
    regressor.add(BatchNormalization())
    regressor.add(LSTM(units=40, return_sequences=True, activation = LeakyReLU(alpha=0.1)))
    regressor.add(BatchNormalization())
    regressor.add(LSTM(units=60, return_sequences=True, activation = LeakyReLU(alpha=0.1)))
    regressor.add(BatchNormalization())
    regressor.add(GRU(units=80, return_sequences=True, activation = LeakyReLU(alpha=0.1)))
    regressor.add(BatchNormalization())
    regressor.add(GRU(units=60, return_sequences=True, activation = LeakyReLU(alpha=0.1)))
    regressor.add(BatchNormalization())
    regressor.add(GRU(units=40, return_sequences=True, activation = LeakyReLU(alpha=0.1)))
    regressor.add(BatchNormalization())
    regressor.add(GRU(units=20, return_sequences=True, activation = LeakyReLU(alpha=0.1)))
    regressor.add(GRU(units=1))
#     regressor.add(RepeatVector(4))

    # regressor.compile(optimizer="adam", loss='mean_squared_error')
    regressor.compile(optimizer="adam", loss='mean_squared_error', metrics=["accuracy"])
    regressor.summary()

    return regressor

In [43]:
for i in range(4):

    print("Lag time = {}".format(i+1))
    ## Time lag
    X1_train, Y1_train= buildTrain(train, pastWeek=i+1, futureWeek=1)

    ## Split date to training & test data
    X1_training = np.round(pd.DataFrame(X1_train[0:int(X1_train.shape[0]*0.8)]),2)
    X1_test = np.round(pd.DataFrame(X1_train[int(X1_train.shape[0]*0.8):]),2)

    Y1_training = np.round(pd.DataFrame(Y1_train[0:int(Y1_train.shape[0]*0.8)]),2)
    Y1_test = np.round(pd.DataFrame(Y1_train[int(Y1_train.shape[0]*0.8):]),2)

    ## Normalize
    print("---Normalize---")
    X1_training_scaled = sc.fit_transform(X1_training)
    X1_test_scaled = sc.transform(X1_test)

    Y1_training_scaled = sc.fit_transform(Y1_training)
    print("The shape of Y1_training: {}".format(Y1_training_scaled.shape))
    Y1_test_scaled = sc.transform(Y1_test)
    print("The shape of Y1_test_scaled: {}".format(Y1_test_scaled.shape))

    ## Reshape the data to 3-dimensions


    X1_training_scaled = X1_training_scaled.reshape(X1_training_scaled .shape[0],X1_training_scaled .shape[1],1)
    Y1_training_scaled = Y1_training_scaled.reshape(Y1_training_scaled.shape[0],Y1_training_scaled.shape[1],1)

    X1_test_scaled = X1_test_scaled.reshape(X1_test_scaled.shape[0],X1_test_scaled.shape[1],1)
    Y1_test_scaled = Y1_test_scaled.reshape(Y1_test_scaled.shape[0],Y1_test_scaled.shape[1],1)

    print("---Reshape the data to 3-dimensions---")
    print("X1_training_scaled:{0}".format(X1_training_scaled.shape))
    print("Y1_train_scaled:{0}".format(Y1_training_scaled.shape))
    print("X1_test_scaled:{0}".format(X1_test_scaled.shape))
    print("Y1_test_scaled:{0}".format(Y1_test_scaled.shape))

    ## Training model
    input_sahpe = X1_training_scaled.shape[1]
    regressor = buildLSTMModel(input_sahpe)
    history = regressor.fit(X1_training_scaled, Y1_training_scaled, epochs=1000, batch_size=4, shuffle=True)
    # indicator = {"Loss":history.history["loss"], "accuracy":history.history["accuracy"]}
    # print(indicator)
    # show_raw_visualization(pd.DataFrame(indicator))

    plt.plot(history.history["loss"])
    plt.plot(history.history["accuracy"])
    plt.legend(labels=['loss','accuracy'],loc='best')
    plt.show()
    ## Evaluate the model
    scores = regressor.evaluate(X1_test_scaled, Y1_test_scaled) 
    print(scores)

Lag time = 1
---Normalize---
The shape of Y1_training: (382, 1)
The shape of Y1_test_scaled: (96, 1)
---Reshape the data to 3-dimensions---
X1_training_scaled:(382, 15, 1)
Y1_train_scaled:(382, 1, 1)
X1_test_scaled:(96, 15, 1)
Y1_test_scaled:(96, 1, 1)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 15, 20)            1760      
_________________________________________________________________
batch_normalization (BatchNo (None, 15, 20)            80        
_________________________________________________________________
lstm_1 (LSTM)                (None, 15, 40)            9760      
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 40)            160       
_________________________________________________________________
lstm_2 (LSTM)                (None, 15, 60)            24240     
_

KeyboardInterrupt: 

[提升深度學習模型的表現，你需要這20個技巧](https://www.itread01.com/content/1545435569.html)

[Reference1](https://ithelp.ithome.com.tw/questions/10199599)
[李鴻毅模型訓練不好怎麼辦](https://baubibi.medium.com/%E9%80%9F%E8%A8%98ai%E8%AA%B2%E7%A8%8B-%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92%E5%85%A5%E9%96%80-%E4%B8%80-68e27912ce30)

--------------------------------

**XGBClassifier for var imp**

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 11 15:33:20 2018

@author: slave1
"""

from sklearn.metrics import confusion_matrix, auc, accuracy_score
import pandas as pd
import numpy as np
import statsmodels.api as sm
import itertools

# ----設定繪圖-------
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus'] = False  
from matplotlib.font_manager import FontProperties 
import seaborn as sns 

In [ ]:
def model(clf, X_train,y_train) :
    model = clf.fit(X_train, y_train)
    
    plot_name="XGboost"
    # y_pred = model.predict(X_train)
    
    
    
    feat_imp = model.feature_importances_

    #取出所有feature的columns name
    feat = X_train.columns.tolist()

    res_data = pd.DataFrame({'Features': feat, 'Importance': feat_imp}).sort_values(by='Importance', ascending=True)
    res_data.plot('Features', 'Importance', kind='barh', title='XGboostPerformance',stacked=True, figsize = (15,10))
    plt.ylabel(plot_name+' Feature Importance Score')
    plt.savefig(plot_name+'.png', dpi=300)
    plt.show()
    
    
    res_data = res_data.sort_values('Importance',ascending=False)
    
    # > 50%
    res_data = res_data[res_data['Importance']>res_data['Importance'].describe()['50%']]
    
    print(res_data)

    # print( "################ summary ################ ")
    # print(confusion_matrix(y_test, y_pred))
    # print("Training Accuracy = {:.3f}".format(model.score(X_train, y_train)))
    # print("Test Accuracy = {:.3f}".format(model.score(X_test, y_test)))
    
    
    # importance
    # print( '------------------ 應注意之變數 ------------------' )
                    
    # print(res_data)
    
    # print( '------------------' +plot_name+ '重要變數表------------------' )
    # res_data.to_excel(plot_name+'重要變數表.xlsx')
    
    
    return res_data

In [ ]:
from xgboost import XGBClassifier

In [ ]:
X1_training = pd.DataFrame(X1_train,columns=['CDCS', 'ADCS', 'CDWS', 'GL', 'NDCS', 'NRBCON', 'SL', 'ZSCS',
'CCSP', 'EURUSD', 'USDCLP', 'USDCNY', 'USDPEN', 'USInflation', 'ChinaInflation'])

Y1_training = pd.DataFrame(Y1_train[:,0],columns=['CCSP'])

In [ ]:
# plot feature importance manually
from numpy import loadtxt
from xgboost import XGBClassifier
from matplotlib import pyplot
from sklearn.datasets import load_iris

# fit model no training data
model = XGBClassifier()

model.fit(X1_training, Y1_training)
# feature importance
print(model.feature_importances_)
# plot
pyplot.bar(range(len(model.feature_importances_)), model.feature_importances_)
pyplot.show()

In [ ]:
res_data = model(
        clf = XGBClassifier(n_estimators = 400, random_state = 0),
        X_train=X1_training, 
        y_train=Y1_training
    )

#n_estimators = 樹的個數, random_state = 0, nthread =使用全部CPU進行並行運算

**Hyperparameter**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense

In [ ]:
# history = model.fit(
#     X1_training, Y1_training, 
#     epochs=50, 
#     batch_size=20, 
#     validation_data = (X1_test, Y1_test), 
#     shuffle=False
# )

In [ ]:
path = "/content/gdrive/My Drive/Colab Notebooks/finalData.csv"
finalData = read(path)
x_train=finalData.drop(["Date"], axis=1)
y_train=np.array(finalData["CCSP"], dtype=int)

In [ ]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

# 選擇要保留的特徵數
select_k = 10

selection = SelectKBest(mutual_info_classif, k=select_k).fit(x_train, y_train)

# 顯示保留的欄位
features = x_train.columns[selection.get_support()]
print(features)

### 被視為較有影響力的input factor
['CDCS', 'CDWS', 'NDCS', 'CCSP', 'EURUSD', 'USDCLP', 'USDCNY', 'USDPEN', 'USInflation', 'ChinaInflation']]